In [16]:
import lxml.html as html
import pandas as pd
import os, glob, re, string
from glob import glob
from bs4 import BeautifulSoup
import logging, importlib, sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from _logging import set_logging
from save_file import save_file

In [17]:
set_logging(logging)

In [18]:
def read_html_file(fname):
    with open (fname, 'r', encoding="utf8") as f:
        doc = BeautifulSoup(f.read(), 'html.parser')
        return doc

In [19]:
def check_job_is_data_science(title):
    re_pattern = re.compile(r'\b(?:data|computer vision|deep learning|machine learning|natural language processing|NLP|AI|A[.]I[.]|artificial)(?<!\w\B)',re.IGNORECASE)
    re_results = re.findall(re_pattern, title)
    logging.info(f"For title {title}, found matching {re_results}")
    flag = len(re_results) > 0
    return flag

In [20]:
stop_words = set(stopwords.words('english'))
files = glob(os.path.join("Files\\Input", '*.html'))
titles = []
ds_titles = []
not_ds_titles = []
selected_job_descriptions = []
for i, fname in enumerate(files):
    message = f"{i+1}. file : {fname}"
    logging.info(message)
    doc = read_html_file(fname)
    title = doc.title.string
    titles.append(title)
    if check_job_is_data_science(title):
        ds_titles.append(title)
        selected_job_descriptions.append((title, doc))
    else:
        not_ds_titles.append(title)


        

2022-12-22 22:41:34,147 | INFO : 1. file : Files\Input\001b92395ed0fb62.html
2022-12-22 22:41:34,154 | INFO : For title Data Scientist - Beavercreek, OH, found matching ['Data']
2022-12-22 22:41:34,155 | INFO : 2. file : Files\Input\00321a48d04fe754.html
2022-12-22 22:41:34,160 | INFO : For title Data Scientist - Seattle, WA 98101, found matching ['Data']
2022-12-22 22:41:34,160 | INFO : 3. file : Files\Input\0079c11b2611349f.html
2022-12-22 22:41:34,169 | INFO : For title Data Scientist - Newark, CA, found matching ['Data']
2022-12-22 22:41:34,170 | INFO : 4. file : Files\Input\007d9d7b5c09d820.html
2022-12-22 22:41:34,175 | INFO : For title Patient Care Assistant / PCA - Med/Surg (Fayette, AL) - Fayette, AL, found matching []
2022-12-22 22:41:34,176 | INFO : 5. file : Files\Input\00bf37ae19c7dfd7.html
2022-12-22 22:41:34,185 | INFO : For title Data Manager / Analyst - Oakland, CA, found matching ['Data']
2022-12-22 22:41:34,186 | INFO : 6. file : Files\Input\0125eabc844281c9.html
202

In [21]:
filename = "titles.csv"
save_file(filename, pd.DataFrame(titles))
filename = "ds_titles.csv"
save_file(filename, pd.DataFrame(ds_titles))
filename = "not_ds_titles.csv"
save_file(filename, pd.DataFrame(not_ds_titles))

Output saved to file Files\Output\titles.csv
Output saved to file Files\Output\ds_titles.csv
Output saved to file Files\Output\not_ds_titles.csv


In [22]:
def tokenise_words(text, stop_words):
    word_tokens = word_tokenize(text)
    # converts the words in word_tokens to lower case and then checks whether
    #they are present in stop_words or not
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    #with no lower case conversion
    filtered_sentence = ""
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence += filtered_sentence + w + " "

    return filtered_sentence

In [23]:
tag_list = ["p", "b", "div", "l1", "h1", "h2", "h3", "h4"]
dict_tag_text = {}
for tag in tag_list:
    dict_tag_text[tag] = ""
dict_job_description_text = {}
all_text = ""

for i, (title, doc) in enumerate(selected_job_descriptions):
    job_description_text = ""
    for tag in tag_list:
        for item in doc.find_all(tag):
            selected_text = item.get_text(separator=" ")
            selected_text = selected_text.translate(str.maketrans('','', string.punctuation))
            selected_text = selected_text.replace("\n", " ")
            selected_text = tokenise_words(selected_text, stop_words)
            dict_tag_text[tag] += selected_text + " "
            job_description_text += selected_text
            all_text += selected_text
    
    dict_job_description_text[i] = (title, job_description_text)

j = 0

MemoryError: 